# Part 3: Single-View Geometry

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [2]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import scipy
import numpy

from PIL import Image
from sympy import Matrix, solve, symbols

# Provided functions

In [17]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [18]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [19]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [26]:
def get_vanishing_point(lines: numpy.ndarray) -> numpy.ndarray:
    """
    Solves for the vanishing point using the user-input lines.
    Input:
        lines: np.ndarray of shape (3, n) where each column denotes the parameters of the line equation
               Line equation: line[0] * x + line[1] * y + line[2] = 0
    Return:
        eigvec: vanishing point
    """
    gram_mat = lines@lines.T
    eigvals, eigvecs = scipy.linalg.eig(a=gram_mat)
    idx = np.argmin(a=eigvals)
    eigvec = eigvecs[:, idx]
    eigvec /= eigvec[-1]

    return eigvec

In [27]:
def get_horizon_line(vpts: numpy.ndarray) -> numpy.ndarray:
    """
    Calculates the ground horizon line.
    Input:
        vpts: vanishing points
    Return:
        line: np.ndarray of shape (3, 1) where each column denotes the parameters of the line equation
              Line equation: line[0] * x + line[1] * y + line[2] = 0
    """
    pt_0 = vpts[:, 0]
    pt_1 = vpts[:, 1]
    line = np.cross(a=pt_0, b=pt_1)
    norm = scipy.linalg.norm(a=line)
    line = line/norm

    return line

In [28]:
def plot_horizon_line(mat: numpy.ndarray, line: numpy.ndarray) -> None:
    """
    Plots the horizon line.
    """
    w = mat.shape[1]
    x = np.arange(w)
    y = (-line[2]-line[0]*x)/line[1]
    plt.figure()
    plt.imshow(X=mat)
    plt.plot(x, y, color="cyan", linestyle='-', linewidth=3)    
    plt.show()

    return

In [42]:
def get_camera_parameters(vpts: numpy.ndarray) -> tuple[float, float, float]:
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    Input:
        vpts: vanishing points
    Return:
        abs(f), px, py
    """
    vp1 = vpts[:, 0].reshape((-1, 1))  # column vector
    vp2 = vpts[:, 1].reshape((-1, 1))
    vp3 = vpts[:, 2].reshape((-1, 1))

    f, px, py= symbols("f, px, py")
    K_inv = Matrix([[1 / f, 0, -px / f], [0, 1 / f, -py / f], [0, 0, 1]])

    eq_0 = vp1.T * K_inv.T * K_inv * vp2
    eq_1 = vp1.T * K_inv.T * K_inv * vp3
    eq_2 = vp2.T * K_inv.T * K_inv * vp3

    f, px, py = solve([eq_0[0], eq_1[0], eq_2[0]], (f, px, py))[0]
    print(f, px, py)

    return abs(f), px, py

In [3]:
def get_rotation_matrix(vpts: numpy.ndarray, f: float, u: float, v: float) -> numpy.ndarray:
    """
    Computes the rotation matrix using the camera parameters.
    """
    # r_i = lambda_i * K^(-1) * v_i
    Z = vpts[:, 0].reshape((-1, 1))  # column vector
    X = vpts[:, 1].reshape((-1, 1))
    Y = vpts[:, 2].reshape((-1, 1))
    
    K = np.array([[f, 0, u], [0, f, v], [0, 0, 1]]).astype(np.float64)
    K_inv = scipy.linalg.inv(a=K)
    r1= K_inv@X
    r2= K_inv@Y
    r3= K_inv@Z
    
    r1 = r1 / scipy.linalg.norm(a=r1)
    r2 = r2 / scipy.linalg.norm(a=r2)
    r3 = r3 / scipy.linalg.norm(a=r3)

    R = np.concatenate((r1, r2, r3), axis=1)

    return R

In [31]:
def estimate_height():
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # <YOUR IMPLEMENTATION>
    pass

# Main function

In [32]:
im = np.asarray(Image.open('images/CSL.jpeg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines)
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 2
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the mi

In [36]:
mat = np.asarray(Image.open('images/CSL.jpeg'))

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts=vpts)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(mat=mat, line=horizon_line)

In [43]:
# im = np.asarray(Image.open('CSL.jpeg'))

# # Part 1
# # Get vanishing points for each of the directions
# num_vpts = 3
# vpts = np.zeros((3, num_vpts))
# for i in range(num_vpts):
#     print('Getting vanishing point %d' % i)
#     # Get at least three lines from user input
#     n, lines, centers = get_input_lines(im)
#     # <YOUR IMPLEMENTATION> Solve for vanishing point
#     vpts[:, i] = get_vanishing_point(lines=lines)
#     # Plot the lines and the vanishing point
#     plot_lines_and_vp(im, lines, vpts[:, i])

# # <YOUR IMPLEMENTATION> Get the ground horizon line
# horizon_line = get_horizon_line()
# # <YOUR IMPLEMENTATION> Plot the ground horizon line
# plot_horizon_line()

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts=vpts)
exit()
# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix()

# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height()


-570.693810138943 239.028795928684 438.534750265308


NameError: name 'R' is not defined